In [7]:
%matplotlib widget
import os

from matplotlib import pyplot as plt
from themachinethatgoesping.echosounders import index_functions
from themachinethatgoesping.echosounders import kongsbergall
from tqdm.auto import tqdm

folders = []
folders.append("../unittest_data")

# list raw data files
files = index_functions.find_files(folders, [".all","wcd"])
files.sort()

# load files' data
fh = kongsbergall.KongsbergAllFileHandler(files)


Found 18 files
indexing files ⠐ 100% :00s<00m:00s] [..0871266855321420.all (1/18)]                               
indexing files ⠠ 100% :00s<00m:00s] [..3008643552583898.wcd (18/18)]                                
indexing files ⢀ 100% :00s<00m:00s] [Found: 729 datagrams in 18 files (10MB)]                                         
Initializing ping interface ⢀ 90% :00s<00m:00s] [Done]                                              


In [2]:
#get file paths from pings
ping = fh.get_pings()[10]

#all files used for a ping (relevant if .all and .wcd are used)
files = ping.file_data.get_file_paths()
print(files)

#primary file is always the .all (except if there is no .all file)
file = ping.file_data.get_primary_file_path()
print(file)




['../unittest_data/kongsberg/a/y/-7609271985000766481.all', '../unittest_data/kongsberg/a/y/-7609271985000766481.wcd']
../unittest_data/kongsberg/a/y/-7609271985000766481.all


In [4]:
#open for a single file
fh = kongsbergall.KongsbergAllFileHandler(file)

indexing file ⠐ 100% :00s<00m:00s]           
indexing file ⠠ 100% :00s<00m:00s] [Found: 127 datagrams]                      
Initializing ping interface ⠐ 0% :00s<00m:00s] [Done]                                              


In [5]:
#display all datagrams
print(fh.datagram_interface.datagrams())

DatagramContainer
#################
-
Time info (Datagrams) 
------------------------ 
- Start time: 13/02/2014 08:12:41.58 
- End time:   13/02/2014 08:12:42.78 
- Sorted:     no                     

 Contained datagrams 
---------------------       
- Total:                                       127 
- Datagrams [PUStatusOutput]:                  1   [31]
- Datagrams [AttitudeDatagram]:                1   [41]
- Datagrams [ClockDatagram]:                   1   [43]
- Datagrams [InstallationParametersStart]:     1   [49]
- Datagrams [RawRangeAndAngle]:                35  [4e]
- Datagrams [PositionDatagram]:                1   [50]
- Datagrams [RuntimeParameters]:               3   [52]
- Datagrams [SoundSpeedProfileDatagram]:       1   [55]
- Datagrams [XYZDatagram]:                     35  [58]
- Datagrams [SeabedImageData]:                 35  [59]
- Datagrams [DepthOrHeightDatagram]:           1   [68]
- Datagrams [NetworkAttitudeVelocityDatagram]: 12  [6e]


In [9]:
#loop through all datagrams
for d in tqdm(fh.datagram_interface.datagrams()):
    print(d.get_datagram_identifier())
    break

  0%|          | 0/729 [00:00<?, ?it/s]

t_KongsbergAllDatagramIdentifier.InstallationParametersStart


In [10]:
#loop through all XYZDatagrams

for d in tqdm(fh.datagram_interface.datagrams("XYZDatagram")):
    print(d.get_datagram_identifier())
    break

  0%|          | 0/93 [00:00<?, ?it/s]

t_KongsbergAllDatagramIdentifier.XYZDatagram


In [11]:
#get the first installationparametersstart datagrams
d  = fh.datagram_interface.datagrams("InstallationParametersStart")[0]
print(d)

InstallationParameters
######################
- bytes:               1073     
- stx:                 0x02     
- datagram_identifier: 0x49     [InstallationParametersStart]
- model_number:        EM2040   
- date:                20220730 [YYYYMMDD]
- time_since_midnight: 74863867 [ms]

 date/time 
-----------  
- timestamp: 1659.214e⁶   [s]
- date:      30/07/2022   [MM/DD/YYYY]
- time:      20:47:43.867 [HH:MM:SS]

 datagram content 
------------------   
- installation_parameters_counter: 6     
- system_serial_number:            2106  
- secondary_system_serial_number:  2086  
- etx:                             0x03  
- checksum:                        63519 

 substructures 
---------------                                                 
- installation_parameters: WLZ=-2.040,SMH=2106,STC=3,S1X=0.000,S1Y=0.000,S1Z=... 

 processed (installation parameter fields) 
------------------------------------------- 
- AHE: 2                                      [Active heave sensor]
- AHS:

In [18]:
#get the data as string (as described in the file format)
print(d.get_active_heave_sensor())

t_KongsbergAllActiveSensor.MotionSensor1


In [19]:
d.get_model_number()

2040

In [22]:
#get runtime parametser
for d in tqdm(fh.datagram_interface.datagrams("RuntimeParameters")):
    print(d.get_datagram_identifier())
    break
print(d)
d.get_mode()

  0%|          | 0/38 [00:00<?, ?it/s]

t_KongsbergAllDatagramIdentifier.RuntimeParameters
RuntimeParameters
#################
- bytes:               52       
- stx:                 0x02     
- datagram_identifier: 0x52     [RuntimeParameters]
- model_number:        EM2040   
- date:                20220730 [YYYYMMDD]
- time_since_midnight: 74864309 [ms]

 date/time 
-----------  
- timestamp: 1659.214e⁶   [s]
- date:      30/07/2022   [MM/DD/YYYY]
- time:      20:47:44.309 [HH:MM:SS]

 datagram content 
------------------        
- ping_counter:                         63074 
- system_serial_number:                 2106  
- operator_station_status:              0     
- processing_unit_status:               0     
- bsp_status:                           0     
- sonar_head_or_transceiver_status:     0     
- mode:                                 129   
- filter_identifier:                    130   
- minimum_depth:                        20    [m]
- maximum_depth:                        100   [m]
- absorption_coefficient: 

129

In [24]:
#raw
print(d.get_transmit_pulse_length())

#processed
print(d.get_transmit_pulse_length_in_seconds())

107
0.00010699999984353781


In [26]:
ping = fh.get_pings()[0]

rp = ping.file_data.get_runtime_parameters()
print(rp)

RuntimeParameters
#################
- bytes:               52       
- stx:                 0x02     
- datagram_identifier: 0x52     [RuntimeParameters]
- model_number:        EM710    
- date:                20120821 [YYYYMMDD]
- time_since_midnight: 61782856 [ms]

 date/time 
-----------  
- timestamp: 1345.569e⁶   [s]
- date:      21/08/2012   [MM/DD/YYYY]
- time:      17:09:42.856 [HH:MM:SS]

 datagram content 
------------------        
- ping_counter:                         62485 
- system_serial_number:                 210   
- operator_station_status:              0     
- processing_unit_status:               0     
- bsp_status:                           0     
- sonar_head_or_transceiver_status:     0     
- mode:                                 129   
- filter_identifier:                    111   
- minimum_depth:                        30    [m]
- maximum_depth:                        250   [m]
- absorption_coefficient:               2407  [0.01 db/km]
- transmit_pulse_l

In [31]:
f'{rp.get_mode():b}'

'10000001'